# Extraction of the parameters of the neural networks generated by NN_generation.ipynb

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Loading one model to obtein the numbers of layers
from keras.models import load_model
model = load_model('E:/Ensemble/Immagini distorte/solo distorsione/epoca1-200/Checkpoint/model_NN0_ep0.hdf5')
num_layers = len(model.layers)

In [3]:
num_layers

6

In [4]:
def data_extraction(model, l):
    """
    input:  model -> network from which to extract the parameters
            l -> layer from which to extract the parameters of the network "model"
            
    output: concatenation -> vector obtained by concatenating weights and biases of layer l:
                             [weights_flatten]U[bias])[layer[l] 
    """
    import h5py
    weights = model.layers[l].get_weights()[0]
    biases = model.layers[l].get_weights()[1]
    concatenation = np.concatenate((weights.flatten(),biases))
    return concatenation

In [5]:
def NeuralNet_matrix(e, num_NN, num_layers):
    """
    input:  e -> epoch that is being investigated
            num_NN -> number of neural networks analyzed
            num_layers -> number of layers making up the networks
    
    output: NNs_matrix -> num_NN x D matrix with D number of parameters (weights and biases) making up the network.
                          Each row is thus formed:
                          ([weights_flatten]U[bias])[layer[1])U([weights_flatten]U[bias])[layer[2])...
    """
    
    NNs_matrix = []
    for i in range(num_NN):
        NN_parametres = np.array([])
        model = load_model('E:\Ensemble\Immagini distorte\solo distorsione\epoca1-200\Checkpoint/model_NN'+str(i)+'_ep'+str(e)+'.hdf5')
        for l in range(1,num_layers,2):                    # 2 is for overcoming the dropout layer                 
            concatenation = data_extraction(model, l)   
            NN_parametres = np.append(NN_parametres,concatenation)
        NNs_matrix.append(NN_parametres)
    NNs_matrix_np = np.array(NNs_matrix)
    return NNs_matrix_np

In [6]:
# import TwoNN di A. Laio
import Two_NN 
T = Two_NN.TwoNN(discard=0.1,block_analysis=False)

# import MLE di Levina e Bickel
import pandas as pd
from geomle import mle

In [9]:
epochs = 200
num_NN = 100

NN_parametres = np.array([])
NNs_matrix = []

# TwoNN result
TNN_res = []                    
TNN_ID = np.zeros(epochs)      # vector containing the IDs estimated by TwoNN for each epoch

# MLE result
MLE_ID = np.zeros(epochs)      # vector containing the IDs estimated by MLE for each epoch



for epoch in range(0,epochs):
    # load Neural Networks parametres
    data_NNs = NeuralNet_matrix(epoch+1, num_NN, num_layers)
    
    # saving the matrix of the networks at the epoch time in format (D x num_NN)
    # to estimate the ID with FastDANCo and MiND in Matlab
    arr_matlab = data_NNs.transpose()
    np.savetxt("E:/Ensemble/Immagini distorte/solo distorsione/epoca1-200/Matlab/mat_ep"+str(epoch+1)+".txt", arr_matlab, delimiter=',')

    # apply TwoNN
    TNN_res = T.fit(data_NNs)
    TNN_ID[epoch] = TNN_res.DimEstimate_
    
    # apply MLE    
    MLE_data = pd.DataFrame(data_NNs)
    MLE_res_temp = mle(MLE_data)[0]
    MLE_ID[epoch] = MLE_res_temp.mean()
    

    
# creating the dataframe with the results just found
Results = pd.DataFrame()
Results['Epoch'] = [e for e in range(0,epochs)]
Results['TwoNN ID'] = TNN_ID
Results['MLE ID'] = MLE_ID

# saving the dataframe in order to update it with FastDANCo and MiND data from Matlab
Results.to_excel("E:/Ensemble/Immagini distorte/solo distorsione/epoca1-200/soloDistorsione_model_sub_ID_finali.xlsx")
